In [ ]:
import sys
assert sys.version_info[0]==3
assert sys.version_info[1] >= 5

from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import pprint
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import nltk
nltk.download('reuters')
from nltk.corpus import reuters
import numpy as np
import random
import scipy as sp
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

START_TOKEN = '<START>'
END_TOKEN = '<END>'

np.random.seed(0)
random.seed(0)

In [ ]:
def read_corpus(category="crude"):
    # Read files from the specified Reuter's category.
    files = reuters.fileids(category)
    return [[START_TOKEN] + [w.lower() for w in list(reuters.words(f))] + [END_TOKEN] for f in files]
reuters_corpus = read_corpus()

In [ ]:
def distinct_words(corpus):
    corpus_words = []
    num_corpus_words = -1
    
    flat=[y for x in corpus for y in x]
    [corpus_words.append(i) for i in flat if i not in corpus_words]
    corpus_words.sort()
    num_corpus_words=len(corpus_words)
    return corpus_words, num_corpus_words

In [ ]:
def compute_co_occurrence_matrix(corpus, window_size=4):
    words, num_words = distinct_words(corpus)
    M = None
    word2Ind = {}
    
    word2Ind = {words[i]: i for i in range(len(words))}
    print(word2Ind)
    M = np.zeros( (num_words, num_words) )
    for j in range (len(corpus)):
        for i in range (len(corpus[j])):
            count=window_size
            oh=1
            for x in range(len(corpus[j]) -  (i+1)):
                if count==0: break
                M[word2Ind[corpus[j][i]]]  [word2Ind[corpus[j][i+oh]]]  =M[word2Ind[corpus[j][i]]]  [word2Ind[corpus[j][i+oh]]]+1
                count=count-1
                oh=oh+1
            count=window_size
            oh=1
            for x in range( len(corpus[j]) -  (len(corpus[j])-corpus[j].index(corpus[j][i]))):
                #
                if count==0: break
                M[word2Ind[corpus[j][i]]]  [word2Ind[corpus[j][i-oh]]]  =M[word2Ind[corpus[j][i]]]  [word2Ind[corpus[j][i-oh]]]+1
                count=count-1
                oh=oh+1

    return M, word2Ind

In [ ]:
def reduce_to_k_dim(M, k=2):
   
    n_iters = 10     # Use this parameter in your call to `TruncatedSVD`
    M_reduced = None
    print("Truncated SVD on %i words:" % (M.shape[0]))

    svd=TruncatedSVD(2)
    M_reduced=svd.fit_transform(M)

    print("Complete")
    return M_reduced

In [ ]:
def plot_embeddings(M_reduced, word2Ind, words):

    for i,type in enumerate(words):
        x = M_reduced[i,0]
        y = M_reduced[i,1]
        plt.scatter(x, y, marker='x', color='red')
        plt.text(x, y, type, fontsize=12)
    plt.show()

In [ ]:
reuters_corpus = read_corpus()
M_co_occurrence, word2Ind_co_occurrence = compute_co_occurrence_matrix(reuters_corpus)
M_reduced_co_occurrence = reduce_to_k_dim(M_co_occurrence, k=2)

# Rescale (normalize) the rows to make them each of unit-length
M_lengths = np.linalg.norm(M_reduced_co_occurrence, axis=1)
M_normalized = M_reduced_co_occurrence / M_lengths[:, np.newaxis] # broadcasting

words = ['barrels', 'bpd', 'ecuador', 'energy', 'industry', 'kuwait', 'oil', 'output', 'petroleum', 'venezuela']
plot_embeddings(M_normalized, word2Ind_co_occurrence, words)